# 카드 추천 AI 만들기

<<진행 순서 가이드>>
주제 : 카드 상품 추천 대화형AI 구현 실습 (고객 맞춤형 카드 추천 챗봇 서비스 개발)
1. 카드 상품 데이터 크롤링
 - 카드고릴라 사이트의 카드 탭에서 신용카드 및 체크카드 데이터 활용 
 - 웹사이트 : https://www.card-gorilla.com/card?cate=CRD
 - 추후 RAG구조 구현시 청킹을 위해 어떤 형태로 데이터를 수집해둘 것인지 판단하여 데이터 수집 진행
2. 가상인물 지정 (3 Case 이상)
 - ex) 20대 취업준비생으로 대중교통으로 매일 학원에 통원하며, 식비를 아끼기 위해 편의점에서 주로 끼니를 해결하고, 애완동물을 기르고 있음.
 - ex) 30대 여성 직장인으로 여행을 좋아해 연 2회 이상 해외여행을 다니는 중, 몸이 자주 아파 병원을 많이 이용함, 자차를 소지함
3. LangChain API를 활용한 대화형 AI 시스템 구축
 - Base 모델은 gpt-4o-mini로 사용
4. 모델 고도화
 - 프롬프트 엔지니어링
 - RAG 구현
5. 모델 평가
 1) base gpt-4o-mini 모델의 응답과 프롬프팅 및 RAG 구성후의 응답을 비교
 2) 페르소나 별 모델에게 추천 받은 카드를 홈페이지에서 검색하여 맞게 추천했는지 사람이 직접 검증(Human Based Evaluation)
 - 평가 횟수는 페르소나별로 최소 10번 이상 진행하여 응답 결과에 대한 만족을 상,중,하로 통계낼 것
 - 즉, temperature를 0.8로 설정하여 동일한 프롬프트로 10개의 서로다른 응답을 생성한 후 상, 중, 하로 통계내기 


※ 발표 참고사항
 - 팀별로 데이터 수집 부터 RAG 구현까지 전체 과정에 대한 코드 리뷰 형식으로 발표
 - 데이터 수집 양, 전처리 형식, 성능이 좋았던 프롬프트 및 RAG기법 및 그에 대한 응답 결과는 발표에 필수적으로 포함시킬 것

## 카드 정보 크롤링

In [5]:
import requests
import time
from bs4 import BeautifulSoup as bs

In [88]:
# 신용카드 idx 수집
url = "https://api.card-gorilla.com:8080/v1/cards"

all_idx = []
page = 1

while True:
    params = {
        'page': page,
        'perPage': 500,  # 최대값 사용
        'cate': 'CRD',
        'is_discon': False  # 단종카드 필터링
    }
    
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()
        
        cards = data.get('data', [])
        
        # 데이터가 없으면 종료
        if not cards:
            print(f"No more data at page {page}")
            break
        
        # idx 추출
        page_ids = [card['idx'] for card in cards]
        all_idx.extend(page_ids)
        
        # 진행상황 출력
        total = data.get('total', '?')
        print(f"Page {page}: {len(page_ids)} cards | Total: {len(all_idx)}/{total}")
        
        # 전체 개수에 도달했는지 확인
        if len(all_idx) >= data.get('total', float('inf')):
            print(f"✓ All pages collected!")
            break
        
        page += 1
        time.sleep(0.5)
        
    except Exception as e:
        print(f"Error at page {page}: {e}")
        break

print(f"\n{'='*50}")
print(f"✓ Total cards collected: {len(all_idx)}")
print(f"✓ Unique cards: {len(set(all_idx))}")
print(f"✓ ID range: {min(all_idx)} ~ {max(all_idx)}")
print(f"✓ Total pages: {page - 1}")

Page 1: 799 cards | Total: 799/799
✓ All pages collected!

✓ Total cards collected: 799
✓ Unique cards: 799
✓ ID range: 8 ~ 2914
✓ Total pages: 0


In [89]:
# 체크카드 idx 수집
url = "https://api.card-gorilla.com:8080/v1/cards"

all_chk = []
page = 1

while True:
    params = {
        'page': page,
        'perPage': 500,  # 최대값 사용
        'cate': 'CHK',
        'is_discon': False  # 단종카드 필터링
    }
    
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        data = response.json()
        
        cards = data.get('data', [])
        
        # 데이터가 없으면 종료
        if not cards:
            print(f"No more data at page {page}")
            break
        
        # idx 추출
        page_ids = [card['idx'] for card in cards]
        all_chk.extend(page_ids)
        
        # 진행상황 출력
        total = data.get('total', '?')
        print(f"Page {page}: {len(page_ids)} cards | Total: {len(all_chk)}/{total}")
        
        # 전체 개수에 도달했는지 확인
        if len(all_chk) >= data.get('total', float('inf')):
            print(f"✓ All pages collected!")
            break
        
        page += 1
        time.sleep(0.5)
        
    except Exception as e:
        print(f"Error at page {page}: {e}")
        break

print(f"\n{'='*50}")
print(f"✓ Total cards collected: {len(all_chk)}")
print(f"✓ Unique cards: {len(set(all_chk))}")
print(f"✓ ID range: {min(all_chk)} ~ {max(all_chk)}")
print(f"✓ Total pages: {page - 1}")

Page 1: 367 cards | Total: 367/367
✓ All pages collected!

✓ Total cards collected: 367
✓ Unique cards: 367
✓ ID range: 279 ~ 2910
✓ Total pages: 0


In [90]:
# 신용카드 필요한 카드 정보 추출
def extract_card_info(idx):
    try:
        response = requests.get(f"https://api.card-gorilla.com:8080/v1/cards/{idx}", timeout=30)
        response.raise_for_status()  # 4xx, 5xx 에러 체크
        response = response.json()
    except requests.exceptions.JSONDecodeError:
        print(f"JSON 아님! 받은 내용: {response.text}\n idx:{idx}")
    except requests.exceptions.RequestException as e:
        print(f"요청 실패: {e}\n idx:{idx} ")

    return {
        'name': response['name'],
        'card_type': '신용카드',
        'brand': [item['name'] for item in response['brand']] if response['brand'] else None,
        'issuer': response['corp']['name'],
        'only_online': response['only_online'],
        'pre_month_money': response['pre_month_money'],     # type int
        'annual_fee': response['annual_fee_basic'],
        'key_benefits': [
            {
                'title': response['key_benefit'][num]['title'],
                'info': response['key_benefit'][num]['comment'],
                'detail': clean_html(response['key_benefit'][num]['info'])
            }
            for num in range(0, len(response['key_benefit'])-1)
        ],
        'top_benefits': [' '.join([bene for bene in response['top_benefit'][i]['tags']]) for i in range(len(response['top_benefit']))]
    }

# html 태그 정리

def clean_html(html_text):
    soup = bs(html_text, 'html.parser')
    
    # 텍스트만 추출
    text = soup.get_text()
    
    return text

In [91]:
# 체크카드 필요한 카드 정보 추출
def extract_card_info_chk(idx):
    try:
        response = requests.get(f"https://api.card-gorilla.com:8080/v1/cards/{idx}", timeout=30)
        response.raise_for_status()  # 4xx, 5xx 에러 체크
        response = response.json()
    except requests.exceptions.JSONDecodeError:
        print(f"JSON 아님! 받은 내용: {response.text}\n idx:{idx}")
    except requests.exceptions.RequestException as e:
        print(f"요청 실패: {e}\n idx:{idx} ")

    return {
        'name': response['name'],
        'card_type': '체크카드',
        'brand': [item['name'] for item in response['brand']] if response['brand'] else None,
        'issuer': response['corp']['name'],
        'only_online': response['only_online'],
        'pre_month_money': response['pre_month_money'],     # type int
        'key_benefits': [
            {
                'title': response['key_benefit'][num]['title'],
                'info': response['key_benefit'][num]['comment'],
                'detail': clean_html(response['key_benefit'][num]['info'])
            }
            for num in range(0, len(response['key_benefit'])-1)
        ],
        'top_benefits': [' '.join([bene for bene in response['top_benefit'][i]['tags']]) for i in range(len(response['top_benefit']))]
    }

# html 태그 정리

def clean_html(html_text):
    soup = bs(html_text, 'html.parser')
    
    # 텍스트만 추출
    text = soup.get_text()
    
    return text

In [92]:
# 신용카드 모든 idx 정보 추출
all_card_info = []

for j in all_idx:
   all_card_info.append(extract_card_info(j))

In [93]:
# 체크카드 모든 idx 정보 추출
all_card_info_chk = []

for j in all_chk:
   all_card_info_chk.append(extract_card_info_chk(j))

In [94]:
# 모든 정보 잘 크롤링됐는지 확인
print(len(all_card_info))
print(len(all_card_info_chk))

799
367


In [95]:
# 신용카드 데이터와 체크카드 데이터 합치기
total_card_info = all_card_info + all_card_info_chk

In [ ]:
# json 형식으로 저장하기
import json

# JSON 파일로 저장
with open('/Users/minjikim/Desktop/sesac/project2/total_card_info.json', 'w', encoding='utf-8') as f:
    json.dump(total_card_info, f, ensure_ascii=False, indent=2)

In [98]:
# 필요없는 key 제거

# JSON 파일 로드
with open("/Users/minjikim/Desktop/sesac/project2/total_card_info_summarized_2.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# 각 카드에서 특정 키 제거
keys_to_remove = ["pre_month_money", "annual_fee", "top_benefits"]

for card in data:
    for key in keys_to_remove:
        card.pop(key, None)  # 키가 없으면 무시

# 수정된 데이터 다시 저장
with open("/Users/minjikim/Desktop/sesac/project2/total_card_info_summarized_3.json", "w", encoding="utf-8") as f:
    json.dump(data, f, ensure_ascii=False, indent=2)

## 혜택 세부사항 요약 AI

In [1]:
from dotenv import load_dotenv
import json

load_dotenv()

True

In [20]:
# 요약 이유: 길고 중복된 문장은 불필요한 “잡음 토큰”이 많아서 의미 중심 좌표가 퍼져버림
# 긴 문장은 임베딩 품질이 낮아지고, 질의랑 덜 맞게 느껴짐

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

# 카드 세부사항 요약해주는 summaraize_detail 함수 정의
def summarize_detail(detail, temperature, model):
    system_prompt = """
당신은 신용카드 혜택 요약 전문가입니다.  
아래의 혜택 상세 설명을 읽고, 숫자·브랜드명·혜택 종류·한도·조건 중심으로 간결하게 요약하세요.  
출력은 문장 요약이 아니라 “정보 요약 문장” 형태로 작성합니다.  

요약 시 다음 기준을 지켜주세요:
1. 혜택의 핵심 정보(할인율, 적립률, 브랜드명)를 빠짐없이 포함할 것
2. 전월실적 및 혜택 한도에 대한 너무 자세한 설명은 제거할 것(전월실적 구간별 혜택한도 등)
3. 불필요한 수식어나 중복된 설명은 제거할 것
4. 혜택이 여러 개인 경우 ‘/'로 구분 (최대 3줄 이내)
5. 문장 끝에는 마침표를 사용하지 않음

예시:
- 스타벅스·이디야 10% 캐시백, 월 최대 3천원  
- Weverse Shop 2% 포인트 적립, 월 최대 5천P  
- 편의점·택시·배달앱 5% 할인 (전월실적 30만원 이상)

---detail---
{detail}
""" 

    prompt = ChatPromptTemplate.from_template(system_prompt)

    # 모델 정의 (gpt-3.5-turbo, temperature=0)
    llm = ChatOpenAI(model=model, temperature=temperature)

    # chain생성
    chain = prompt | llm | StrOutputParser() 

    # 응답 출력 
    response = chain.invoke({"detail": detail})

    return response 

In [21]:
# 함수 잘 돌아가는지 체크
detail = "월납요금(공과금) 10% 할인서비스- 전기요금 /도시가스요금/ SKT, LG U+, KT 통신요금 할인 * 인터넷/집전화/이동통신/결합상품 포함- 일 1회 할인 적용- 1회 이용 금액 5만원까지 할인 적용 (1회 최대 5천원 할인)할인한도안내- 전월실적 30만원~50만원 : 3천원- 전월실적 50만원 ~100만원 : 7천원- 전월실적 100만원 이상 : 1만원- 서비스 대상 거래건 중 신한카드 전표 매입 순서대로 결제일 할인이 적용됩니다.- 월납(공과금) 할인서비스는 전월 이용금액에 따라 제공된 월 할인 한도 내에서 서비스가 제공됩니다.- 신규 발급 회원에 대해서는 카드사용 등록월의 익월말(등록월+1개월)까지 3천원의 할인한도가 제공됩니다.- 전월 이용실적은 일시불+할부 금액 기준이며 (전월 할인거래 실적 포함) 교통이용금액은 전전월 이용금액 기준, 해외이용- 금액은 매입일자를 기준으로 적용됩니다- 월 기준 : 매월 1일 ~ 말일Powered by Froala Edito"

print(summarize_detail(detail, 0.2, "gpt-4o-mini"))

- 전기요금·도시가스요금·SKT·LG U+·KT 통신요금 10% 할인, 1회 최대 5천원 할인 (전월실적 30만원 이상)  
- 신규 발급 회원 3천원 할인 한도 제공


In [ ]:
# 상위 5개 카드만 체크
with open("/Users/minjikim/Desktop/sesac/project2/total_card_info.json", "r", encoding="utf-8") as f:
    cards = json.load(f)

for card in cards[0:5]:
    for benefit in card.get("key_benefits", []):
        detail_text = benefit.get("detail", "")
        if detail_text:
            # summarize_detail 함수 사용
            summary = summarize_detail(detail_text, 0.2, "gpt-4o-mini")
            benefit["detail"] = summary  # 원래 detail 자리 덮어쓰기

print("모든 key_benefits.detail 요약 완료")
cards[0:5]

In [ ]:
# 모든 데이터 요약 > 1시간 반 넘게 걸림 미친 것 같음
with open("/Users/minjikim/Desktop/sesac/project2/total_card_info.json", "r", encoding="utf-8") as f:
    cards = json.load(f)

for card in cards:
    for benefit in card.get("key_benefits", []):
        detail_text = benefit.get("detail", "")
        if detail_text:
            # summarize_detail 함수 사용
            summary = summarize_detail(detail_text, 0.2, "gpt-4o-mini")
            benefit["detail"] = summary  # 원래 detail 자리 덮어쓰기

print("모든 key_benefits.detail 요약 완료")

# 새로운 json 파일로 저장
with open("total_card_info_summarized.json", "w", encoding="utf-8") as f:
    json.dump(cards, f, ensure_ascii=False, indent=2)

In [ ]:
# 너무 느리다... 더 빠른 요약 방법? > 20분 걸림
from concurrent.futures import ThreadPoolExecutor, as_completed

# 캐시 딕셔너리
summary_cache = {}

# 요약 함수 (캐시 적용)
def summarize_detail_cached(detail, temperature=0.2, model="gpt-4o-mini"):
    if not detail:
        return ""
    if detail in summary_cache:
        return summary_cache[detail]

    system_prompt = """
당신은 신용카드 혜택 요약 전문가입니다.  
아래의 혜택 상세 설명을 읽고, 숫자·브랜드명·혜택 종류·한도·조건 중심으로 간결하게 요약하세요.  
출력은 문장 요약이 아니라 “정보 요약 문장” 형태로 작성합니다.  

요약 시 다음 기준을 지켜주세요:
1. 혜택의 핵심 정보(할인율, 적립률, 브랜드명)를 빠짐없이 포함할 것
2. 전월실적 및 혜택 한도에 대한 너무 자세한 설명은 제거할 것(전월실적 구간별 혜택한도 등)
3. 불필요한 수식어나 중복된 설명은 제거할 것
4. 혜택이 여러 개인 경우 ‘/'로 구분 (최대 3줄 이내)
5. 문장 끝에는 마침표를 사용하지 않음

---detail---
{detail}
"""
    prompt = ChatPromptTemplate.from_template(system_prompt)
    llm = ChatOpenAI(model=model, temperature=temperature)
    chain = prompt | llm | StrOutputParser()
    summary = chain.invoke({"detail": detail})
    summary_cache[detail] = summary
    return summary

# JSON 파일 불러오기
with open("/Users/minjikim/Desktop/sesac/project2/total_card_info.json", "r", encoding="utf-8") as f:
    cards = json.load(f)

# ThreadPoolExecutor로 병렬 처리
max_workers = 10  # 동시에 처리할 스레드 수
with ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = []
    for card in cards:
        for benefit in card.get("key_benefits", []):
            detail_text = benefit.get("detail", "")
            if detail_text:
                futures.append((benefit, executor.submit(summarize_detail_cached, detail_text)))

    # 결과 가져오기
    for benefit, future in futures:
        benefit["detail"] = future.result()

print("모든 key_benefits.detail 요약 완료")

# 새로운 JSON 파일로 저장
with open("total_card_info_summarized_2.json", "w", encoding="utf-8") as f:
    json.dump(cards, f, ensure_ascii=False, indent=2)

모든 key_benefits.detail 요약 완료


## RAG

In [2]:
from langchain.document_loaders import JSONLoader
from langchain.docstore.document import Document
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import LLMChain

In [3]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

### 데이터 요약 X

In [ ]:
# JSON 파일 로드
with open("/Users/minjikim/Desktop/sesac/project2/total_card_info.json", "r", encoding="utf-8") as f:
    data_json = json.load(f)

# 카드 단위 Document 생성 (page_content는 문자열, metadata는 dict)
documents = []
for card in data_json:
    card_text = json.dumps(card, ensure_ascii=False)  # JSON 객체 → 문자열
    documents.append(Document(page_content=card_text, metadata={"card_name": card.get("name","")}))

print(f"총 {len(documents)}개의 Document 생성 완료!")

# 임베딩 생성
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")  # 필요에 따라 모델 선택

# Chroma 벡터스토어 생성
vectordb = Chroma.from_documents(       # 카드 단위로 청킹해도 토큰이 3만자를 넘어서 임베딩 불가
    documents=documents,
    embedding=embeddings,
    collection_name="card_collection"
)
# 5. 검색
retriever = vectordb.as_retriever(search_kwargs={"k": 30})  # 검색기 생성, 몇 개의 청크 뽑을 건지 지정
result = retriever.invoke("30대 초반, 5살 자녀를 둔 엄마. 국내 전용 카드를 발급받으려고 하며, 롯데백화점 문화센터 강좌를 즐겨들음. 주말 마다 강좌를 끝내고 가족들과 TGI FRIDAY에서 외식을 함. 집 앞 GS칼텍스 주유소를 애용하며 CMA 저축 등 실용적이고 가족 중심의 소비 패턴을 가진 라이프스타일임.")
# 사용자 쿼리 입력 -> 해당 쿼리와 유사도가 높은 청크를 찾아준다

print(len(result))  # 몇 개의 요소들이 뽑혔는지
result[0]    # 가장 유사도가 높은 청크

### 데이터 요약 O

In [4]:
import json

In [10]:
# JSON 파일 로드
with open("/Users/minjikim/Desktop/sesac/project2/total_card_info_summarized_3.json", "r", encoding="utf-8") as f:
    data_json = json.load(f)

# 혜택 단위 Document 생성 (page_content는 문자열, metadata는 dict)
documents = []
for card in data_json:
    card_name = card.get("name", "")
    issuer = card.get("issuer", "")
    card_type = card.get("card_type", "")
    brand = ", ".join(card.get("brand", [])) if card.get("brand") else "국내전용"

    # 카드 내 혜택 하나하나를 독립 문서로
    for benefit in card.get("key_benefits", []):
        info = benefit.get("info", "")
        detail = benefit.get("detail", "")

        # 카드 정보 + 혜택 설명을 자연어로 연결
        text = f"""
        카드명: {card_name}
        발급사: {issuer}
        카드종류: {card_type}
        브랜드: {brand}
        혜택 요약: {info}
        혜택 상세: {detail}
        """
        documents.append(Document(page_content=text.strip(), metadata={"card_name": card_name}))

print(f"총 {len(documents)}개의 혜택 Document 생성 완료")
# 청킹
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,   # 한 chunk당 몇 자?
    chunk_overlap=200  # 청킹 사이 몇 자 중복?
)

chunked_documents = []
for doc in documents:
    chunks = splitter.split_text(doc.page_content)
    for chunk in chunks:
        chunked_documents.append(Document(page_content=chunk, metadata=doc.metadata))

# 임베딩 및 벡터스토어
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")  # 필요에 따라 모델 선택
batch_size = 200
vectorstore = None

for i in range(0, len(chunked_documents), batch_size):
    batch = chunked_documents[i:i + batch_size]
    print(f"임베딩 진행 중... {i} ~ {i + len(batch)} / {len(chunked_documents)}")
    vectorstore = Chroma.from_documents(
            documents=batch,
            embedding=embeddings,
            # persist_directory="/Users/minjikim/Desktop/sesac/project2/chroma2",
            # collection_name="card"
        )

# 5. 검색
retriever = vectorstore.as_retriever(search_kwargs={"k": 10})  # 검색기 생성, 몇 개의 청크 뽑을 건지 지정
result = retriever.get_relevant_documents("20대 여성 A씨는 배달의 민족을 이용하여 음식을 주문하는 걸 즐긴다. 온라인 쇼핑을 즐기며 대중교통을 자주 이용한다.") # 사용자 쿼리 입력 -> 해당 쿼리와 유사도가 높은 청크를 찾아준다

print(len(result))  # 몇 개의 요소들이 뽑혔는지
result[0:5]    # 가장 유사도가 높은 청크

총 5173개의 혜택 Document 생성 완료
임베딩 진행 중... 0 ~ 200 / 5173


Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


임베딩 진행 중... 200 ~ 400 / 5173


Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


임베딩 진행 중... 400 ~ 600 / 5173


...

Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


임베딩 진행 중... 5000 ~ 5173 / 5173


Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


10


[Document(metadata={'card_name': 'NH20 해봄카드'}, page_content='카드명: NH20 해봄카드\n        발급사: NH농협카드\n        카드종류: 신용카드\n        브랜드: Mastercard, JCB\n        혜택 요약: 대중교통 10% 할인\n        혜택 상세: 대중교통(버스/지하철) 10% 청구할인 / 월 이용금액 2만원 이상 시 적용 / 후불교통 이용건에 한함'),
 Document(metadata={'card_name': '국민행복카드(비씨)'}, page_content='카드명: 국민행복카드(비씨)\n        발급사: NH농협카드\n        카드종류: 신용카드\n        브랜드: UnionPay\n        혜택 요약: 임신출산, 보육료/유아학비 등 지원\n        혜택 상세: 국민행복카드: A-Type 5% 청구할인(병·의원, 조산원, 산후조리원) / 주요 온라인 쇼핑몰 5% 청구할인(G마켓, 옥션 등) / 주요 커피 전문점 20% 청구할인(스타벅스, 커피빈 등) / 주요 패밀리레스토랑 10% 청구할인(아웃백, TGIF 등) / 이동통신요금 자동이체 1천원 청구할인 / 전국 시내버스, 지하철 5% 청구할인 / 통합 할인한도 최대 3만원  \nB-Type 5% 청구할인(어린이집, 유치원) / 주요 커피 전문점 20% 청구할인(스타벅스, 커피빈 등) / 주요 패밀리레스토랑 10% 청구할인(아웃백, TGIF 등) / 이동통신요금 자동이체 1천원 청구할인 / 전국 시내버스, 지하철 5% 청구할인 / 놀이공원 50% 현장할인(롯데월드, 에버랜드 등) / 통합 할인한도 최대 3만원  \nC-Type 기본 적립 0.2~0.8% 에코머니 포인트 / 추가 적립 1~4% 에코머니 포인트(의료, 육아 업종) / 대중교통 10~20% 에코머니 포인트 적립 / KTX, 고속버스 5% 에코머니 포인트 적립 / 대중교통 통합 적립한도 최대 1만점'),
 Document(met

In [11]:
result

[Document(metadata={'card_name': 'NH20 해봄카드'}, page_content='카드명: NH20 해봄카드\n        발급사: NH농협카드\n        카드종류: 신용카드\n        브랜드: Mastercard, JCB\n        혜택 요약: 대중교통 10% 할인\n        혜택 상세: 대중교통(버스/지하철) 10% 청구할인 / 월 이용금액 2만원 이상 시 적용 / 후불교통 이용건에 한함'),
 Document(metadata={'card_name': '국민행복카드(비씨)'}, page_content='카드명: 국민행복카드(비씨)\n        발급사: NH농협카드\n        카드종류: 신용카드\n        브랜드: UnionPay\n        혜택 요약: 임신출산, 보육료/유아학비 등 지원\n        혜택 상세: 국민행복카드: A-Type 5% 청구할인(병·의원, 조산원, 산후조리원) / 주요 온라인 쇼핑몰 5% 청구할인(G마켓, 옥션 등) / 주요 커피 전문점 20% 청구할인(스타벅스, 커피빈 등) / 주요 패밀리레스토랑 10% 청구할인(아웃백, TGIF 등) / 이동통신요금 자동이체 1천원 청구할인 / 전국 시내버스, 지하철 5% 청구할인 / 통합 할인한도 최대 3만원  \nB-Type 5% 청구할인(어린이집, 유치원) / 주요 커피 전문점 20% 청구할인(스타벅스, 커피빈 등) / 주요 패밀리레스토랑 10% 청구할인(아웃백, TGIF 등) / 이동통신요금 자동이체 1천원 청구할인 / 전국 시내버스, 지하철 5% 청구할인 / 놀이공원 50% 현장할인(롯데월드, 에버랜드 등) / 통합 할인한도 최대 3만원  \nC-Type 기본 적립 0.2~0.8% 에코머니 포인트 / 추가 적립 1~4% 에코머니 포인트(의료, 육아 업종) / 대중교통 10~20% 에코머니 포인트 적립 / KTX, 고속버스 5% 에코머니 포인트 적립 / 대중교통 통합 적립한도 최대 1만점'),
 Document(met

## 최종 프롬프트

### 페르소나 1

In [12]:
# 시스템/유저 프롬프트 정의
system_prompt = """
당신은 유능한 카드사 직원입니다. 
고객이 제공하는 정보를 고려하여 **필요 혜택**을 많이 받을 수 있는 2개의 카드를 추천순대로 추천해 주세요.
**적용되는 필요 혜택이 많을수록** 순위가 올라갑니다.

# 꼭 지켜야만하는 규칙
추천하는 두 개의 카드는 **서로 다른 카드**여야만 합니다.
당신의 답변은 반드시 question과 context에 기반해야 하며, context 외 정보를 추측하거나 보충하지 않습니다.

# 출력 템플릿 예시
1순위
추천 카드명(name)
카드 정보: 카드타입/은행명/카드브랜드(국내전용이라면 국내전용)/온라인 발급 전용 유무
추천 이유: 추천 이유 1줄 (여러 혜택을 강조하세요)
혜택
    - question에 따라 고객에게 적용되는 혜택 1
    - question에 따라 고객에게 적용되는 혜택 2 (혜택 1과 구분되는 다른 혜택)

2순위
추천 카드명(name)
카드 정보: 카드타입/은행명/카드브랜드(국내전용이라면 국내전용)/온라인 발급 전용 유무
추천 이유: 추천 이유 1줄 (여러 혜택을 강조하세요)
혜택
    - question에 따라 고객에게 적용되는 혜택 1
    - question에 따라 고객에게 적용되는 혜택 2 (혜택 1과 구분되는 다른 혜택)

사용자 질의를 보고 context를 참고하여 답변해주세요.
신용카드를 원하는지, 체크카드를 원하는지 구분하고 신용카드를 원하면 신용카드만, 체크카드를 원하면 체크카드만 추천하세요.

--- 사용자 질의 ---
{question}

# context 구성
참고할 context에 대해서는 아래처럼 해석해 주세요
card_type: 카드가 신용카드인지, 체크카드인지
issuer: 은행명
brand: 카드가 어떤 브랜드인지. 브랜드가 빈값이면 국내전용카드
only_online: 온라인발급전용이면 true, 아니면 false
key_benefits: 카드가 제공하는 혜택

--- context ---
{context}
"""

# ChatPromptTemplate.from_messages() 사용
final_prompt = ChatPromptTemplate.from_messages([
    ("system", system_prompt)
])

# 모델 설정
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 체인 구성 (| 파이프라인 연산자 사용)
chain = final_prompt | model | StrOutputParser()

# 완성 프롬프트 
question = """

"""

# 응답 생성
response = chain.invoke({
    "question": "20대 여성 A씨는 배달의 민족을 이용하여 음식을 주문하는 걸 즐긴다. 온라인 쇼핑을 즐기며 대중교통을 자주 이용한다. A씨를 위한 카드는?",
    "context": result
})

print(response)


1순위  
추천 카드명: 기후동행카드(신용)  
카드 정보: 신용카드/NH농협카드/국내전용/false  
추천 이유: 배달의 민족 이용 시 10% 청구할인과 대중교통 할인 혜택을 동시에 제공하여 A씨의 생활에 최적화된 카드입니다.  
혜택  
    - 배달의 민족 10% 청구할인 (건당 1만원 이상 시 최대 2,000원 할인)  
    - 대중교통 10% 청구할인  

2순위  
추천 카드명: NH20 해봄카드  
카드 정보: 신용카드/NH농협카드/Mastercard, JCB/false  
추천 이유: 온라인 쇼핑몰에서 10% 할인과 대중교통 할인 혜택을 제공하여 A씨의 다양한 소비 패턴을 지원합니다.  
혜택  
    - 온라인 쇼핑몰 10% 청구할인 (G마켓, 옥션 등)  
    - 대중교통 10% 청구할인  


### 페르소나 2

### 페르소나 3